In this assignment, you will be implementing a GPT model and train it using CLM objective.
 * If you get stuck at something or need more clarrifications, you may refer to : https://github.com/karpathy/minGPT/blob/master/mingpt/model.py

 * We will be using ReLU activation function instead of GELU.

 * As usual, let us install the required libraries

 * **Note** that if you are not getting the exact loss values as mentioned in this notebook, that is absolutely fine. Just see whether your implementation overfits the given toy-and-tiny paragraph!

# Installation


In [78]:
!pip install torchdata==0.6.0 # to be compatible with torch 2.0
!pip install portalocker==2.0.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 80.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

* See [here](https://github.com/pytorch/text) for compatability

In [79]:
!pip install -U torchtext==0.15.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.4 MB/s eta 0:00:00
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.17.1
    Uninstalling torchtext-0.17.1:
      Successfully uninstalled torchtext-0.17.1


# Imports

In [1]:
import torch
from torch import Tensor

import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.nn.functional import one_hot

import torch.optim as optim

#text lib
import torchtext

# tokenizer
from torchtext.data.utils import get_tokenizer

#build vocabulary
from torchtext.vocab import vocab
from torchtext.vocab import build_vocab_from_iterator

# get input_ids (numericalization)
from torchtext.transforms import VocabTransform

# get embeddings
from torch.nn import Embedding

from  pprint import pprint
from yaml import safe_load
import copy
import numpy as np
import math

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Load the dataset for LM modeling

 * We use a simple tokenizer and put

In [3]:
batch_size = 10

In [4]:
class Tokenizer(object):

  def __init__(self,text):
    self.text = text
    self.word_tokenizer = get_tokenizer(tokenizer="basic_english",language='en')
    self.vocab_size = None

  def get_tokens(self):
    for sentence in self.text.strip().split('\n'):
      yield self.word_tokenizer(sentence)

  def build_vocab(self):
    v = build_vocab_from_iterator(self.get_tokens(),
                                  min_freq=1,specials=['<unk>','<start>','<end>'])
    v.set_default_index(v['<unk>']) # index of OOV
    self.vocab_size = len(v)
    return v

  def token_ids(self):
    v = self.build_vocab()
    vt = VocabTransform(v)
    num_tokens = len(self.word_tokenizer(self.text))
    max_seq_len = np.ceil(num_tokens/batch_size)
    data = torch.zeros(size=(1,num_tokens))
    data = vt(self.word_tokenizer(self.text))
    data = torch.tensor(data,dtype=torch.int64)
    return data.reshape(batch_size,torch.tensor(max_seq_len,dtype=torch.int64))

In [5]:
text = """Best known for the invention of Error Correcting Codes, he was a true polymath who applied his mathematical and problem-solving skills to numerous disciplines.
Reflecting on the significant benefits I received from Hamming, I decided to develop a tribute to his legacy. There has not been a previous biography of Hamming, and the few articles about him restate known facts and assumptions and leave us with open questions.
One thought drove me as I developed this legacy project: An individual's legacy is more than a list of their attempts and accomplishments. Their tribute should also reveal the succeeding generations they inspired and enabled and what each attempted and achieved.
This book is a unique genre containing my version of a biography that intertwines the story "of a life" and a multi-player memoir with particular events and turning points recalled by those, including me, who he inspired and enabled.
Five years of research uncovered the people, places, opportunities, events, and influences that shaped Hamming. I discovered unpublished information, stories, photographs, videos, and personal remembrances to chronicle his life, which helped me put Hamming's
legacy in the context I wanted.The result demonstrates many exceptional qualities, including his noble pursuit of excellence and helping others. Hamming paid attention to the details, his writings continue to influence, and his guidance is a timeless gift to the world.
This biography is part of """

In [6]:
Tk = Tokenizer(text)

In [7]:
x_raw = Tk.token_ids()
print(x_raw.shape)

torch.Size([10, 26])


In [8]:
# let us display the first 10 tokens of the vocabulary
v = Tk.build_vocab()
pprint(v.vocab.get_itos()[0:10])

['<unk>', '<start>', '<end>', ',', 'and', '.', 'the', 'a', 'of', 'to']


* Create the input_ids and Labels from the raw input sequence

In [9]:
bs,raw_seq_len = x_raw.shape
x = torch.empty(size=(bs,raw_seq_len+2),dtype=torch.int64)
x[:,1:-1] =x_raw

# insert the index of special tokens
x[:,0] = torch.full(size=(1,batch_size),fill_value=v.vocab.get_stoi()['<start>'])
x[:,-1] = torch.full(size=(1,batch_size),fill_value=v.vocab.get_stoi()['<end>'])

#Quickly check implem
v = Tk.build_vocab()
words = []
for idx in x[0,:]:
  words.append(v.vocab.get_itos()[idx.item()])
print(' '.join(words))

<start> best known for the invention of error correcting codes , he was a true polymath who applied his mathematical and problem-solving skills to numerous disciplines . <end>


In [10]:
# labels are just the input_ids shifted by right
bs,seq_len = x.shape
y = torch.empty(size=(bs,seq_len),dtype=torch.int64)
y[:,0:-1] = copy.deepcopy(x[:,1:])

#ignore the index of padded tokens while computing loss
y[:,-1] = torch.full(size=(1,batch_size),fill_value=-100)

# Configuration

In [11]:
vocab_size = Tk.vocab_size
seq_len = x.shape[1]
embed_dim = 32
dmodel = embed_dim
dq = torch.tensor(4)
dk = torch.tensor(4)
dv = torch.tensor(4)
heads = torch.tensor(8)
d_ff = 4*dmodel

* Define all the sub-layers (mhma,ffn) in the transformer blocks
* Seed for $W_Q,W_K,W_V,W_O$, 43, 44 and 45, 46, respectively
* Seed for ffn $W_1,W_2$,  47 and 48. There are no biases
* Seed for output layer 49

In [12]:
class MHMA(nn.Module):

    def __init__(self, dmodel, dq, dk, dv, heads, mask = None):
        super(MHMA, self).__init__()
        self.heads = heads
        self.dq = dq
        self.dk = dk
        self.dv = dv
        self.dmodel = dmodel
        self.mask = mask
        self.dropout = nn.Dropout(p = 0.1)

        self.WQ = nn.Parameter(self.init_param(heads * dq, dmodel, seed = 43))
        self.WK = nn.Parameter(self.init_param(heads * dk, dmodel, seed = 44))
        self.WV = nn.Parameter(self.init_param(heads * dv, dmodel, seed = 45))
        self.WO = nn.Parameter(self.init_param(dmodel, heads * dv, seed = 46))
        self.scaling_factor = math.sqrt(dk)

    def init_param(self, *size, seed):
        torch.manual_seed(seed)
        return torch.randn(size)

    def forward(self, x, mask=None):
        BS, T, _ = x.shape
        Q = torch.matmul(x, self.WQ.T).view(BS, T, self.heads, -1).transpose(1, 2)
        K = torch.matmul(x, self.WK.T).view(BS, T, self.heads, -1).transpose(1, 2)
        V = torch.matmul(x, self.WV.T).view(BS, T, self.heads, -1).transpose(1, 2)

        attention_scores = torch.matmul(Q, K.transpose(-2, -1)) / self.scaling_factor

        if mask is None:
            mask = torch.triu(torch.ones((T, T), device = x.device, dtype = torch.bool), diagonal = 1)
            mask = mask[None, None, :, :].expand(BS, self.heads, T, T)
            attention_scores = attention_scores.masked_fill(mask, float('-inf'))
        else :
            attention_scores = attention_scores + mask

        attention = F.softmax(attention_scores, dim=-1)
        # attention = self.dropout(attention)

        out = torch.matmul(attention, V).transpose(1, 2).contiguous().view(BS, T, -1)
        out = torch.matmul(out, self.WO.T)
        return out


class FFN(nn.Module):

    def __init__(self, dmodel, d_ff):
        super(FFN, self).__init__()
        torch.manual_seed(47)
        self.linear1 = nn.Linear(dmodel, d_ff)
        self.dropout = nn.Dropout(p = 0.1)
        torch.manual_seed(48)
        self.linear2 = nn.Linear(d_ff, dmodel)

        self._reset_parameters()

    def _reset_parameters(self):
        nn.init.kaiming_normal_(self.linear1.weight, mode = 'fan_out', nonlinearity = 'relu')
        nn.init.zeros_(self.linear1.bias)
        nn.init.kaiming_normal_(self.linear2.weight, mode = 'fan_out', nonlinearity = 'relu')
        nn.init.zeros_(self.linear2.bias)

    def forward(self, x):
        '''
        Input  : Size [BS, T, dmodel]
        Output : Size [BS, T, dmodel]
        '''
        x = self.linear1(x)
        x = F.relu(x)
        # x = self.dropout(x)
        out = self.linear2(x)
        return out


class PredictionHead(nn.Module):

    def __init__(self, dmodel, vocab_size):
        super(PredictionHead, self).__init__()
        torch.manual_seed(49)
        self.linear = nn.Linear(dmodel, vocab_size)
        self.dropout = nn.Dropout(p = 0.1)
        self._reset_parameters()

    def _reset_parameters(self):
        nn.init.normal_(self.linear.weight, std = 0.02)
        nn.init.zeros_(self.linear.bias)

    def forward(self, representations):
        '''
        Input  :  Size  [BS, T, dmodel]
        Output :  Size  [BS, T, vocab_size]
        Note   : Do not apply the softmax. Just return the output of linear transformation
        '''
        out = self.linear(representations)
        # out = self.dropout(out)
        return out


class PositionalEncoding(nn.Module):

    def __init__(self, d_model, max_len = 500):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype = torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        x: Tensor, shape [batch_size, seq_len, embedding_dim]
        """
        x = x + self.pe[:, :x.size(1)]
        return x

In [13]:
class DecoderLayer(nn.Module):

  def __init__(self, dmodel, dq, dk, dv, d_ff, heads, mask = None):
      super(DecoderLayer, self).__init__()
      self.mhma = MHMA(dmodel, dq, dk, dv, heads, mask = mask)
      self.layer_norm_1 = torch.nn.LayerNorm(dmodel)
      self.layer_norm_2 = torch.nn.LayerNorm(dmodel)
      self.ffn = FFN(dmodel, d_ff)

  def forward(self, dec_rep):
      norm_mhma_in = self.layer_norm_1(dec_rep)
      mhma_out = self.mhma(norm_mhma_in) + dec_rep

      norm_ffn_in = self.layer_norm_2(mhma_out)
      ffn_out = self.ffn(norm_ffn_in) + mhma_out

      return ffn_out

In [14]:
class Embed(nn.Module):

    def __init__(self, vocab_size, embed_dim):
        super(Embed, self).__init__()
        torch.manual_seed(70)
        self.embed = nn.Embedding(vocab_size, embed_dim)
        self.pe = PositionalEncoding(embed_dim)
        self.dropout = nn.Dropout(p = 0.1)

    def forward(self, x):
        out = self.embed(x)
        out = self.pe(out)  # Applying positional encodings after embedding
        out = self.dropout(out)
        return out

In [15]:
class Decoder(nn.Module):

    def __init__(self, vocab_size, dmodel, dq, dk, dv, d_ff, heads, mask, num_layers = 1):
        super(Decoder,self).__init__()
        self.embed_lookup = Embed(vocab_size, embed_dim)
        self.dec_layers = nn.ModuleList(copy.deepcopy(DecoderLayer(dmodel, dq, dk, dv, d_ff, heads, mask)) for i in range(num_layers))
        self.predict = PredictionHead(dmodel, vocab_size)

    def forward(self,input_ids):
        out = self.embed_lookup(input_ids)
        for dec_layer in self.dec_layers:
          out = dec_layer(out)
        out = self.predict(out)

        return out

In [16]:
model = Decoder(vocab_size, dmodel, dq, dk, dv, d_ff, heads, mask = None)

In [17]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
#optimizer = optim.SGD(model.parameters(), lr = 0.01)

In [18]:
def train(input_ids, labels, epochs = 1000):
  model.train()
  for epoch in range(epochs):
    optimizer.zero_grad()
    out = model(input_ids)
    loss = criterion(out.transpose(1, 2), labels)
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:
      print(f"Epoch: {epoch}, Loss: {loss.item()}")

In [19]:
# run the model for 10K epochs
train(x, y, 10000)

Epoch: 0, Loss: 10.453071594238281
Epoch: 100, Loss: 2.214897394180298
Epoch: 200, Loss: 1.5005929470062256
Epoch: 300, Loss: 1.2446670532226562
Epoch: 400, Loss: 0.9041963815689087
Epoch: 500, Loss: 0.8813911080360413
Epoch: 600, Loss: 0.7321356534957886
Epoch: 700, Loss: 0.5720810294151306
Epoch: 800, Loss: 0.42648574709892273
Epoch: 900, Loss: 0.46503564715385437
Epoch: 1000, Loss: 0.3856365978717804
Epoch: 1100, Loss: 0.47888827323913574
Epoch: 1200, Loss: 0.3263309597969055
Epoch: 1300, Loss: 0.2964624762535095
Epoch: 1400, Loss: 0.32006534934043884
Epoch: 1500, Loss: 0.2963918447494507
Epoch: 1600, Loss: 0.29907286167144775
Epoch: 1700, Loss: 0.28135383129119873
Epoch: 1800, Loss: 0.22392211854457855
Epoch: 1900, Loss: 0.3539230227470398
Epoch: 2000, Loss: 0.23849190771579742
Epoch: 2100, Loss: 0.2545629143714905
Epoch: 2200, Loss: 0.22775694727897644
Epoch: 2300, Loss: 0.3332796096801758
Epoch: 2400, Loss: 0.2699735164642334
Epoch: 2500, Loss: 0.2886599898338318
Epoch: 2600, Los

The loss is about 0.09 after 10K epochs

# Generate text

In [25]:
def generate(model, prompt = '<start>', max_words = 10):
    model.eval()
    # Ensure prompt is a list of words
    if isinstance(prompt, str):
        prompt = [prompt]  # Convert to list if it's a single word

    # Initialize generated sequence with the prompt
    generated_words = prompt

    # Convert prompt words to their corresponding indices
    vocab = Tk.build_vocab()
    stoi = vocab.get_stoi()
    input_ids = [stoi[word] if word in stoi else stoi['<unk>'] for word in prompt]

    # Convert list of input IDs into a tensor
    input_tensor = torch.tensor(input_ids).unsqueeze(0)

    for _ in range(max_words):
        # Predict the next word
        with torch.no_grad():
            output = model(input_tensor)

        # Get the last predicted set of logits and select the word with the highest score
        logits = output[0, -1, :]
        predicted_token_id = logits.argmax(dim=-1).item()
        predicted_word = vocab.get_itos()[predicted_token_id]

        # Break if the model generates an end token
        if predicted_word == '<end>':
            break

        # Append the predicted word to the generated sequence and update the input tensor
        generated_words.append(predicted_word)
        input_ids.append(predicted_token_id)
        input_tensor = torch.tensor([input_ids])  # Update the input for the next prediction

    # Join the list of generated words into a single string
    generated_text = ' '.join(generated_words).replace('<start>', '').strip()
    return generated_text

In [26]:
generate(model, prompt = '<start>', max_words = 25)

'biography that intertwines the story of a life and a multi-player memoir with particular events and turning points recalled by those , including me ,'

* Note the model has memorized the sentence from the training set. Given the start token, if your implementation reproduce a sentence as is in the training set, then your implementation is likely to be correct.
* Suppose the prompt is `<start> best known`, then we expect the model to produce the first sentence as is

In [27]:
generate(model, prompt = ['<start>','best','known'], max_words = 25)

'best known for the invention of error correcting codes , he was a true polymath who applied his mathematical and problem-solving skills to numerous disciplines .'

* Change the prompt

In [28]:
generate(model, prompt = ['<start>','reflecting','on'], max_words = 25)

'reflecting on the significant benefits i received from hamming , i decided to develop a tribute to his legacy . there has not been a previous'